In [97]:
#*****************************************************************************
#**
#**  Pandas Homework: Pandas, Pandas, Pandas
#**
#**   Heroes of Pymoli
#**
#**     Author: George Alonzo
#**   Due Date: Sept 25, 2021
#**
#*****************************************************************************

import pandas as pd
import numpy as np

purchase_file = "Resources/purchase_data.csv"
purchase_file_df = pd.read_csv(purchase_file)

In [98]:
# PLAYER COUNT
print("PLAYER COUNT")

player_num_unique = len(purchase_file_df["SN"].unique())
# Add to a dataframe for better display
player_count_df = pd.DataFrame({"Total Players (Unique)": [player_num_unique]})
player_count_df.style.set_properties(**{'text-align': 'left'})

PLAYER COUNT


,Total Players (Unique)
0,576


In [99]:
# PURCHASING ANALYSIS (TOTAL)
print("*** THIS IS OK *** PURCHASING ANALYSIS (TOTAL)")

item_unique = len(purchase_file_df["Item Name"].unique())
avg_price = purchase_file_df["Price"].mean()
purchases_num = purchase_file_df["Purchase ID"].count()
tot_revenue = purchase_file_df["Price"].sum()

# Add to a dataframe for better display
purchasing_analysis_df = pd.DataFrame({"Number of Unique Items": [item_unique], "Average Price": [avg_price],
                                       "Number of Purchases": [purchases_num], "Total Revenue": [tot_revenue]})

purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].map("${:.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].map("${:,.2f}".format)
purchasing_analysis_df.style.set_properties(**{'text-align': 'left'})

*** THIS IS OK *** PURCHASING ANALYSIS (TOTAL)


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [102]:
# GENDER DEMOGRAPHICS
print("GENDER DEMOGRAPHICS")

#Create new dataframe with distinct Screen Names for demographics
gender_distinct_df = purchase_file_df[["Gender","SN"]].drop_duplicates()

distinct_tot = gender_distinct_df["SN"].count()
gender_counts = gender_distinct_df["Gender"].value_counts()
gender_pcts = (gender_distinct_df["Gender"].value_counts() / distinct_tot) *100

# Add to a dataframe for better display
gender_demo_df = pd.DataFrame({"Total Count": gender_counts, "Percentage": gender_pcts})

gender_demo_df["Percentage"] = gender_demo_df["Percentage"].map("{:.2f}%".format)
gender_demo_df.style.set_properties(**{'text-align': 'left'})

GENDER DEMOGRAPHICS


,Total Count,Percentage
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [101]:
# PURCHASING ANALYSIS (GENDER)
print("PURCHASING ANALYSIS (GENDER)")

purchases_df = purchase_file_df[["Gender","Price"]]
purchases_df.head()

purchases_group = purchases_df.groupby(["Gender"]).count()
purchases_group.columns = ["Purchase Count"]
purchases_group["Average Purchase Price"] = purchases_df.groupby(["Gender"]).mean()
purchases_group["Total Purchase Price"] = purchases_df.groupby(["Gender"]).sum()
purchases_group["Avg Total Purchase Price"] = purchases_group["Total Purchase Price"] / gender_counts

purchases_group["Average Purchase Price"] = purchases_group["Average Purchase Price"].map("${:,.2f}".format)
purchases_group["Total Purchase Price"] = purchases_group["Total Purchase Price"].map("${:,.2f}".format)
purchases_group["Avg Total Purchase Price"] = purchases_group["Avg Total Purchase Price"].map("${:,.2f}".format)
purchases_group.head().style.set_properties(**{'text-align': 'left'})

PURCHASING ANALYSIS (GENDER)


,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase Price
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [103]:
# AGE DEMOGRAPHICS
print("AGE DEMOGRAPHICS")

age_demo_df = purchase_file_df[["SN","Age"]].drop_duplicates()
del age_demo_df["SN"]

age_bins = [0,9,14,19,24,29,34,39,999]
age_groups =["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
age_demo_df["Age Group"] = pd.cut(age_demo_df["Age"], age_bins, labels=age_groups, include_lowest=True)

age_group = age_demo_df.groupby("Age Group").count()
age_group.columns = ["Total Count"]
age_group["Percentage of Players"] = (age_group["Total Count"] / player_num_unique) *100

age_group["Percentage of Players"] = age_group["Percentage of Players"].map("{:.2f}%".format)
age_group.style.set_properties(**{'text-align': 'left'})

AGE DEMOGRAPHICS


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [91]:
# PURCHASING ANALYSIS (AGE)
print("*** IN PROGRESS *** PURCHASING ANALYSIS (AGE)")


# Add bins to the original purchase_file data frame
purchase_file_df["Age Ranges"] = pd.cut(purchase_file_df["Age"], age_bins, labels=age_groups, include_lowest=True)
purchase_file_df.head()

sn_distinct_df = purchase_file_df[["Age Ranges","SN"]].drop_duplicates()
sn_distinct_df.head()

sn_distinct_counts = sn_distinct_df.groupby(["Age Ranges"]).count()
sn_distinct_counts

#age_distinct_counts = sn_distinct_counts["SN"].count()
#age_distinct_counts

#age_purchases_df = purchase_file_df[["Age Ranges","Price"]]
#age_purchases_df.head()

#age_purchases_group = age_purchases_df.groupby(["Age Ranges"]).count()
#age_purchases_group.columns = ["Purchase Count"]
#age_purchases_group["Average Purchase Price"] = age_purchases_df.groupby(["Age Ranges"]).mean()
#age_purchases_group["Total Purchase Value"] = age_purchases_group["Purchase Count"] * age_purchases_group["Average Purchase Price"]
#age_purchases_group["Avg Total Purchase Price"] = age_purchases_group["Total Purchase Value"] / sn_distinct_counts

#age_purchases_group

*** IN PROGRESS *** PURCHASING ANALYSIS (AGE)


,SN
Age Ranges,
<10,17
10-14,22
15-19,107
20-24,258
25-29,77
30-34,52
35-39,31
40+,12


In [104]:
# TOP SPENDERS
print("TOP SPENDERS")

spenders_df = purchase_file_df[["SN", "Price"]]
spenders_df.head()

spenders_group = spenders_df.groupby(["SN"]).count()
spenders_group.columns = ["Purchase Count"]
spenders_group["Average Purchase Price"] = spenders_df.groupby(["SN"]).mean()
spenders_group["Total Purchase Value"] = spenders_group["Purchase Count"] * spenders_group["Average Purchase Price"]
spenders_group = spenders_group.sort_values("Total Purchase Value", ascending=False)
spenders_group["Average Purchase Price"] = spenders_group["Average Purchase Price"].map("${:,.2f}".format)
spenders_group["Total Purchase Value"] = spenders_group["Total Purchase Value"].map("${:,.2f}".format)

spenders_group.head().style.set_properties(**{'text-align': 'left'})

TOP SPENDERS


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [106]:
# MOST POPULAR ITEMS
print("MOST POPULAR ITEMS")

item_df = purchase_file_df[["Item ID", "Item Name", "Price"]]
item_df.head()

item_group = item_df.groupby(["Item ID","Item Name"]).count()
item_group.columns = ["Purchase Count"]
item_group["Item Price"] = item_df.groupby(["Item ID","Item Name"]).mean()

item_group["Total Purchase Value"] = item_group["Purchase Count"] * item_group["Item Price"]
item_group = item_group.sort_values("Purchase Count", ascending=False)
item_group["Item Price"] = item_group["Item Price"].map("${:,.2f}".format)
item_group["Total Purchase Value"] = item_group["Total Purchase Value"].map("${:,.2f}".format)
item_group.head().style.set_properties(**{'text-align': 'left'})

MOST POPULAR ITEMS


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [107]:
print("*** OK, BUT DO I NEED ALL THE CODE? *** MOST PROFITABLE ITEMS")
print("*** HOW TO LEFT-JUSTIFY ITEM NAME? *** MOST PROFITABLE ITEMS")

item_df = purchase_file_df[["Item ID", "Item Name", "Price"]]
item_df.head()

item_group = item_df.groupby(["Item ID","Item Name"]).count()
item_group.columns = ["Purchase Count"]
item_group["Item Price"] = item_df.groupby(["Item ID","Item Name"]).mean()

item_group["Total Purchase Value"] = item_group["Purchase Count"] * item_group["Item Price"]
item_group = item_group.sort_values("Total Purchase Value", ascending=False)
item_group["Item Price"] = item_group["Item Price"].map("${:,.2f}".format)
item_group["Total Purchase Value"] = item_group["Total Purchase Value"].map("${:,.2f}".format)
item_group.head().style.set_properties(**{'text-align': 'left'})

*** OK, BUT DO I NEED ALL THE CODE? *** MOST PROFITABLE ITEMS
*** HOW TO LEFT-JUSTIFY ITEM NAME? *** MOST PROFITABLE ITEMS


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
